In [240]:
%pylab qt4

import os
import vtk
from PyQt4 import QtGui, QtCore
import numpy as np
from math import radians, degrees
from glob import glob
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA, RandomizedPCA
from scipy.interpolate import splprep, splev
from matplotlib import colors
import matplotlib.cm as cmx
from matplotlib.path import Path
from matplotlib.collections import LineCollection

import helpers.classes as ch
import helpers.windows as wh
import helpers.features as fh
import helpers.display as dh
import helpers.geometry as gh
reload(ch)
reload(wh)
reload(fh)
reload(dh)
reload(gh)

BASE_PATH = os.getcwd()
print("Current base path: {0}".format(BASE_PATH))
DATA_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_filtered_and_registered_outline/'
COMPARED_CLASSES = [ 2, 3 ]

STEP_SIZE = 1 # Degrees
EXTENSION_STEP = 0.0025
WINDOW_SIZE = 0.25
NUM_SPLINE_POINTS_IN_OUTLINE = 250
NUM_SPLINE_POINTS_IN_WINDOW = 25
HIGH_RES_AVERAGE_COUNT = 500

def extract_outline_part(spline_params, degrees):
    tck, u = spline_params
    source = np.array([0.0, 0.0])
    ray = np.array(gh.pol2cart(2, radians(degrees)))
    total_space = np.linspace(0, 1, NUM_SPLINE_POINTS_IN_OUTLINE)
    
    coords = splev(total_space, tck)
    num_total_spline_points = len(coords[0])
    total_spline = np.zeros((num_total_spline_points, 2))
    total_spline[:, 0] = coords[0]
    total_spline[:, 1] = coords[1]
    
    param_for_spline = None
    for i in range(0, num_total_spline_points):
        j = i+1 if i+1 < num_total_spline_points else 0
        intersect = gh.seg_intersect(source, ray, total_spline[i, :], total_spline[j, :])
        if intersect is not None:
            dist_from_i = np.linalg.norm(intersect - total_spline[i, :])
            norm_dist_from_i = dist_from_i / np.linalg.norm(total_spline[j, :] - total_spline[i, :])
            
            if total_space[i] == 0:
                param_for_spline = total_space[j]
            if total_space[j] == 0:
                param_for_spline = total_space[i]
            else:
                param_for_spline = total_space[i] * (1-dist_from_i) + total_space[j] * dist_from_i
            break
    
    if param_for_spline is None:
        raise Exception('No intersection found')
    
    current_window_size = 0
    window_width = 0
    while window_width < WINDOW_SIZE:
        current_window_size = current_window_size + EXTENSION_STEP
        
        window_space = np.linspace(param_for_spline-current_window_size, param_for_spline+current_window_size, NUM_SPLINE_POINTS_IN_WINDOW)
        if np.any(window_space < 0):
            window_space[window_space < 0] = window_space[window_space < 0] + 1
        if np.any(window_space > 1):
            window_space[window_space > 1] = window_space[window_space > 1] - 1

        coords = splev(window_space, tck)
        window_spline = np.zeros((len(coords[0]), 2))
        window_spline[:, 0] = coords[0]
        window_spline[:, 1] = coords[1]
    
        window_width = np.cumsum(np.linalg.norm(window_spline - np.roll(window_spline, -1, axis=0), axis=1))[-2]
    return window_spline

def extract_spline(points, num_points=NUM_SPLINE_POINTS_IN_WINDOW):
    y = points[:,0].flatten()
    x = points[:,1].flatten()
    
    tck, u = splprep([y, x], s=0)
    coords = splev(np.linspace(0, 1, num_points-1), tck)
    
    spline_points = np.zeros((len(coords[0]), 2))
    spline_points[:, 0] = coords[0]
    spline_points[:, 1] = coords[1]
    
    return spline_points

def extract_spline_params(points, num_points=NUM_SPLINE_POINTS_IN_WINDOW):
    distances = np.linalg.norm(points - np.roll(points, -1), axis=1)
    distances = np.cumsum(distances / np.sum(distances))
    distances = np.append([ 0.0 ], distances)
    
    y = np.append(points[:,0].flatten(), [points[0,0]])
    x = np.append(points[:,1].flatten(), [points[0,1]])
    
    tck, u = splprep([y, x], s=0, u=distances)
    
    return tck, u

def train_svc(features, classes):
    clf = LinearSVC()
    clf.fit(features, classes)
    
    return clf

def get_recall(svc, features, known_classes):
    predicted_classes = svc.predict(features)
    return float(np.count_nonzero(predicted_classes == known_classes)) / float(known_classes.shape[0])

def get_margin(svc):
    return 2 / np.linalg.norm(svc.coef_)

def map_to_display_mode(definition, closed=False):
    label, cls, points = definition
    if closed:
        edges = np.zeros((points.shape[0], 2), dtype=np.int)
        edges[:,0] = range(0, points.shape[0])
        edges[:,1] = range(1, points.shape[0]+1)
        edges[-1,1] = 0
    else:
        edges = np.zeros((points.shape[0]-1, 2), dtype=np.int)
        edges[:,0] = range(0, points.shape[0]-1)
        edges[:,1] = range(1, points.shape[0])

    return {
        'label': label,
        'class': cls,
        'points': points,
        'edges': edges
    }
   
def feature_flatten_splines(outline_points, splines):
    return np.array([ s.flatten() for s in outline_points ])

def feature_use_distance_to_center(outline_points, splines):
    return np.array([ np.array([ np.linalg.norm(p) for p in o ]) for o in outline_points ])

def feature_use_dist_center_and_curvature(outline_points, splines):
    def curvature(x, y):
        dalpha = np.pi/1000
        xd1 = np.gradient(x)
        xd2 = np.gradient(xd1)
        yd1 = np.gradient(y)
        yd2 = np.gradient(yd1)
        return np.abs(xd1*yd2 - yd1*xd2) / np.power(xd1**2 + yd1**2, 3./2)
    
    distances = feature_use_distance_to_center(outline_points, splines)
    distances = fh.normalize(distances)
    curvature = np.array([curvature(o[:, 0], o[:, 1]) for o in outline_points])
    curvature = fh.normalize(curvature)
    
    features = np.hstack((distances, curvature))
    
    return features

def feature_use_deviation_from_mean(outline_points, splines):
    mean = np.mean(outline_points, axis=0)
    features = []
    for o in outline_points:
        features.append((o - mean).flatten())
    return features

def wrap_with_pca(fn):
    def feature_fn(outline_points, splines):
        features = fn(outline_points, splines)
        pca = PCA(n_components=4)
        reduced = pca.fit_transform(features)
        return reduced
    return feature_fn

def register_single_point(outlines, degrees, feature_fn=wrap_with_pca(feature_use_distance_to_center), display=False):
    outlines = ch.filter_by_classes(outlines, COMPARED_CLASSES)
    labels = [ o['label'] for o in outlines ]
    classes = np.array([ ch.get_class(o['filename'])[0] for o in outlines ])
    splines = [ extract_spline_params(o['points']) for o in outlines ]
    outline_points = [ extract_outline_part(s, degrees) for s in splines ]
    
    if display:
        dh.outlines(map(map_to_display_mode, zip(labels, classes, outline_points)), title='All local outlines', color_by_class=True)
        tmp = np.array(outline_points)
        mean_parts = [
            np.mean(tmp[classes == COMPARED_CLASSES[0], :, :], axis=0),
            np.mean(tmp[classes == COMPARED_CLASSES[1], :, :], axis=0)
        ]
        class_names = [ ch.get_class_name(COMPARED_CLASSES[0]), ch.get_class_name(COMPARED_CLASSES[0]) ]
        dh.outlines(map(map_to_display_mode, zip(class_names, COMPARED_CLASSES, mean_parts)), title='Mean local outlines', color_by_class=True)
    
    features = feature_fn(outline_points, splines)
    svc = train_svc(features, classes)
    
    recall = get_recall(svc, features, classes)
    margin = get_margin(svc)
    rm = recall*margin
    #print("Angle: {angle}, R*M: {rm}, Recall: {recall}, Margin: {margin}".format(angle=degrees, recall=recall, margin=margin, rm=rm))
    
    return {
        'angle': degrees,
        'rm': rm,
        'recall': recall,
        'margin': margin
    }

def get_averages(outlines):
    averages = []
    for cls in COMPARED_CLASSES:
        splines = np.array([
            extract_spline(o['points'], num_points=HIGH_RES_AVERAGE_COUNT)
            for o in outlines if o['class'] == cls
        ])
        averages.append(np.mean(splines, axis=0))
    
    
    return zip([ ch.get_class_name(c) for c in COMPARED_CLASSES ], COMPARED_CLASSES, averages)

def show_averages(outlines):
    averages = get_averages(outlines)
    dh.outlines([ map_to_display_mode(s, closed=True) for s in averages])

def show_averages_with_margin(outlines, results):
    angles = np.array([ m['angle'] for m in results ])
    separabilities = fh.normalize(np.array([ m['rm'] for m in results ]))
    
    current_outline = None
    points = vtk.vtkPoints()
    colors = vtk.vtkUnsignedCharArray()
    linesPolyData = vtk.vtkPolyData()
    def get_outline(factor):
        return np.multiply(outlines[0][2], factor) + np.multiply(outlines[1][2], (1 - factor))
    def get_colors(points):
        colors = []
        for point in points:
            rho, phi = gh.cart2pol(point[0], point[1])
            phi = degrees(phi) + 360 if phi < 0 else degrees(phi)
            closest = np.argmin(np.abs(angles - phi))
            separability = separabilities[closest]
            colors.append(cmx.gnuplot(separability))
        return colors
    def initialize_actor():
        outlinePoints = get_outline(0.5)
               
        vertices = vtk.vtkCellArray()
        for i, point in enumerate(outlinePoints):
            points.InsertNextPoint([ point[1], point[0], 1.0 ])
            vertex = vtk.vtkVertex()
            vertex.GetPointIds().SetId(0, i); 
            vertices.InsertNextCell(vertex); 
        
        lines = vtk.vtkCellArray()
        for i in range(0, outlinePoints.shape[0]):
            line = vtk.vtkLine()
            line.GetPointIds().SetId(0, i)
            line.GetPointIds().SetId(1, i+1 if i+1 < outlinePoints.shape[0] else 0)
            lines.InsertNextCell(line)

        raw_colors = get_colors(outlinePoints)
        colors.SetNumberOfComponents(3)
        colors.SetName("Colors")
        for color in raw_colors:
            colors.InsertNextTuple3(int(color[0] * 255), int(color[1] * 255), int(color[2] * 255))

        linesPolyData.SetPoints(points)
        linesPolyData.SetLines(lines)
        linesPolyData.SetVerts(vertices)
        linesPolyData.GetPointData().SetScalars(colors)

        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(linesPolyData)

        actor = vtk.vtkActor()
        actor.GetProperty().SetLineWidth(3.0)
        actor.SetMapper(mapper)
        
        return actor
    
    def initialize_angle_label_actor():
        txt = vtk.vtkTextActor()
        txt.SetInput("Angle: ")
        txtprop=txt.GetTextProperty()
        txtprop.SetFontFamilyToArial()
        txtprop.SetFontSize(18)
        txtprop.SetColor(0, 0, 0)
        txt.SetDisplayPosition(50,50)
        return txt
    
    def change_value(slider_value):
        try:
            factor = float(slider_value) / 100
            outline = get_outline(factor)
            raw_colors = get_colors(outline)
            for i, point in enumerate(outline):
                points.SetPoint(i, point[1], point[0], 1.0)
                colors.SetTuple3(i, int(raw_colors[i][0] * 255), int(raw_colors[i][1] * 255), int(raw_colors[i][2] * 255))
            linesPolyData.Modified()
            window.vtkWidget.GetRenderWindow().Render()
        except Exception, e:
            print(e)
            
    def on_mouse_move(obj, ev):
        try:
            mouse_position = obj.GetEventPosition()
            if mouse_position:
                window.ren.SetDisplayPoint((mouse_position[0], mouse_position[1], 0))
                window.ren.DisplayToWorld()
                x,y,z,w = window.ren.GetWorldPoint()
                rho, phi = gh.cart2pol(y, x)
                phi = degrees(phi) + 360 if phi < 0 else degrees(phi)
                angleActor.SetInput("Angle: {0}".format(int(phi)))
                window.vtkWidget.GetRenderWindow().Render()
        except Exception, e:
            print(e)
        
    window = wh.VTKWindow(title='Comparison Results')
    outlineActor = initialize_actor()
    angleActor = initialize_angle_label_actor()
    window.iren.AddObserver('MouseMoveEvent', on_mouse_move, 0.0)
    wh.show_window(window)
    window.slider = QtGui.QSlider(QtCore.Qt.Horizontal)
    window.slider.setMinimum(0)
    window.slider.setMaximum(100)
    window.slider.setSliderPosition(50)
    window.vl.addWidget(window.slider)
    window.slider.valueChanged[int].connect(change_value)
    window.render_actors([ outlineActor, angleActor ])

Populating the interactive namespace from numpy and matplotlib
Current base path: /home/stefan/Dropbox/Masterarbeit


`%matplotlib` prevents importing * from pylab and numpy


In [21]:
filenames = glob(DATA_PATH + '*.npz')

outlines = []
for f in filenames:
    basename = os.path.basename(f)
    label = ''.join([i if ord(i) < 128 else ' ' for i in basename])
    outline = np.load(f)
    cls = ch.get_class(basename)
    outlines.append({
        'label': label,
        'class': cls[0],
        'class_label': cls[1],
        'filename': basename,
        'edges': outline['edges'],
        'points': outline['points']
    })

In [211]:
show_averages(outlines)

In [13]:
dh.outlines(outlines, color_by_class=True)

In [167]:
results = []
for angle in range(1, 360, STEP_SIZE):
    results.append(register_single_point(outlines, angle))
for r in results:
    print(r)

{'rm': 1.2571078800499336, 'margin': 2.2856506909998791, 'angle': 1, 'recall': 0.55}
{'rm': 1.4192297817039479, 'margin': 2.5804177849162686, 'angle': 2, 'recall': 0.55}
{'rm': 1.0797670191173345, 'margin': 1.9632127620315172, 'angle': 3, 'recall': 0.55}
{'rm': 0.89747923252204631, 'margin': 1.3807372808031482, 'angle': 4, 'recall': 0.65}
{'rm': 1.1357193783636685, 'margin': 2.5238208408081522, 'angle': 5, 'recall': 0.45}
{'rm': 1.6345800894271512, 'margin': 2.9719637989584564, 'angle': 6, 'recall': 0.55}
{'rm': 1.3919094357706563, 'margin': 2.3198490596177606, 'angle': 7, 'recall': 0.6}
{'rm': 1.3040236306357225, 'margin': 2.6080472612714449, 'angle': 8, 'recall': 0.5}
{'rm': 1.2480618946611814, 'margin': 2.2692034448385114, 'angle': 9, 'recall': 0.55}
{'rm': 1.5541534814259386, 'margin': 2.5902558023765643, 'angle': 10, 'recall': 0.6}
{'rm': 1.4599985026682958, 'margin': 2.4333308377804932, 'angle': 11, 'recall': 0.6}
{'rm': 0.97431665126394684, 'margin': 1.7714848204799032, 'angle':

In [208]:
register_single_point(outlines, 245, display=True)

{'angle': 245,
 'margin': 11.132447637247106,
 'recall': 0.55,
 'rm': 6.1228462004859088}

In [241]:
show_averages_with_margin(get_averages(outlines), results)